# import

# mount

In [ ]:

import pandas as pd
import ujson as json
import re
import networkx as nx
import matplotlib.pyplot as plt
from google.colab import drive
import csv

In [ ]:
pip install ujson

     |████████████████████████████████| 194kB 674kB/s 
  Stored in directory: /root/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
Successfully built ujson


In [ ]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


# reading jsons

In [ ]:
reddit_data_paths_proED = ['/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_proED_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_proED_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_edFood_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_edFood_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_proanaweightloss_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_proanaweightloss_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_thinspo_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_thinspo_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_thinspocommunity_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_thinspocommunity_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_trueThinspo_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_trueThinspo_coms.json',
                    ]

reddit_data_paths_proRecov = ['/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_edAnonymous_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_edAnonymous_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_bingeEatingDisorder_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_bingeEatingDisorder_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_eating_disorders_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_eating_disorders_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_edsupport_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_edsupport_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_bulimia_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_bulimia_coms.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_myProAna_subs.json',
                    '/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_3/reddit_data/prep_myProAna_coms.json',
                    ]

#reddit_data_paths_proED = reddit_data_paths[:int(len(reddit_data_paths)/10)]

_

_

_

# Names





*   **reddit_data_paths_proED**: list of csvs of pro ED content
*   **reddit_data_paths_proRecov**: list of csvs of pro recovery content
*   **df_all_posts_proED**: all posts from ***pro ED subreddits***
*   **df_all_comments_proRecov**: all comments from ***pro recovery subreddits***
*   **post_comments_proED**: post_id -> [ comment_ids...] proED
*   **post_comments_proRecov**: post_id -> [ comment_ids...] preRecov
*   **comment_replies_proED** : comment_id -> [ replies_ids...]
*   **comment_replies_proRecov** : comment_id -> [ replies_ids...]
*   **author_posts_proED**: author_id -> [ post_ids ...]
*   **author_posts_proRecov**: author_id -> [ post_ids ...]
*   **author_comments_proED**: author_id -> [comment_ids ... ]
*   **author_comments_proRecov**: author_id -> [comment_ids ... ]


_

_

_

# helper functions

### extract columns

In [ ]:
def json_to_df(path, type):
  if ( type == "post"):
    return pd.read_json(path)[["author", "id","title", "selftext", "num_comments", "score","created_utc", "subreddit"]]
  elif ( type == "comment"):
    return pd.read_json(path)[["id", "author", "created_utc", "link_id", "parent_id", "score", "body", "subreddit"]]

### merge all posts and all comments into **TWO** dfs

In [ ]:
def merge_dataframes(reddit_data_paths):


  df_all_posts    = json_to_df(reddit_data_paths[0], "post")
  df_all_comments = json_to_df(reddit_data_paths[1], "comment")

  for i in range(2, len(reddit_data_paths), 2):
    #read
    df_posts    = json_to_df(reddit_data_paths[i], "post")
    df_comments = json_to_df(reddit_data_paths[i+1], "comment")
    #append
    df_all_posts.append(df_posts)
    df_all_comments.append(df_comments)

  return df_all_posts, df_all_comments

### is it a comment / post ( t1 / t2) ?

In [ ]:
def get_tx(str):
  return str[:2]

In [ ]:
def remove_tx(str):
   return str[3:]

In [ ]:
def comment_or_reply(comment):
  parent_tx = get_tx(comment["parent_id"])
  #t3_ for links
  if ( parent_tx == "t3"):
    return "comment"
  #t1_ for comments
  elif ( parent_tx == "t1" ):
    return "reply"

  return None

### initialize dicts with ids as keys

In [ ]:
def initialize_lists(df_posts , df_comments, post_comments, comment_replies, author_posts, author_comments):

  post_comments   = {}
  comment_replies = {}
  author_posts    = {}
  author_comments = {}

  #from posts
  for ind, row in df_posts.iterrows():
    post_comments[row["id"]]    = []
    author_posts[row["author"]] = []

  #from comments
  for ind, row in df_comments.iterrows():
    comment_replies[row["id"]]     = []
    author_comments[row["author"]] = []

    #if it s a reply  --> add its parent to the comments
    if (comment_or_reply(row) == "reply"):
      comment_replies[remove_tx(row["parent_id"])] = []

    #if it s a comment linked directly to a post
    #--> add the parent (post) itself to the posts list
    elif(comment_or_reply(row) == "comment"):
      post_comments[remove_tx(row["parent_id"])] = []


  return post_comments, comment_replies, author_posts, author_comments

### for each id asign list of values

In [ ]:
#asigning to each post   its comments
#asigning to each author his things
#each post has many comments
#each author has many comments/posts
def asign_child_to_parent(df_posts, df_comments, post_comments, comment_replies, author_posts, author_comments):

  #asign post to author
  for ind, row in df_posts.iterrows():
    author_posts[row["author"]].append(row["id"])


  for ind, row in df_comments.iterrows():
    #asign comment to author
    author_comments[row["author"]].append(row["id"])


    #asign reply(comment) to comment(post)
    if (comment_or_reply(row) == "reply"):  #if it s a reply  --> add it to replies list

      if ( remove_tx(row["parent_id"]) not in comment_replies):
        comment_replies[remove_tx(row["parent_id"])] = []

      comment_replies[remove_tx(row["parent_id"])].append(row["id"])

    post_comments[remove_tx(row["link_id"])].append(row["id"])



  return post_comments, comment_replies, author_posts, author_comments

### transition to user's relationships

In [ ]:
def get_commenter_repliers_proED(author_comments, comment_replies):
  commenter_repliers = {}
  for auth_id , comments in author_comments.items():
    commenter_repliers[auth_id] = []
    for c_id in comments:
      for r_id in comment_replies[c_id]:
          commenter_repliers[auth_id].append(df_all_comments_proED.loc[r_id].author)
  return commenter_repliers

In [ ]:
def get_poster_commenters_proED(author_posts, post_comments):
  poster_commenters = {}
  for auth_id , posts in author_posts.items():
    poster_commenters[auth_id] = []
    for p_id in posts:
      for c_id in post_comments[p_id]:
          poster_commenters[auth_id].append(df_all_comments_proED.loc[c_id].author)
  return poster_commenters

### Same for pro recovery

In [ ]:
def get_commenter_repliers_proRecov(author_comments, comment_replies):
  commenter_repliers = {}
  for auth_id , comments in author_comments.items():
    commenter_repliers[auth_id] = []
    for c_id in comments:
      for r_id in comment_replies[c_id]:
          commenter_repliers[auth_id].append(df_all_comments_proRecov.loc[r_id].author)
  return commenter_repliers

In [ ]:
def get_poster_commenters_proRecov(author_posts, post_comments):
  poster_commenters = {}
  for auth_id , posts in author_posts.items():
    poster_commenters[auth_id] = []
    for p_id in posts:
      for c_id in post_comments[p_id]:
          poster_commenters[auth_id].append(df_all_comments_proRecov.loc[c_id].author)
  return poster_commenters

_

_

_

_

# main()



1.   chargin dicts containing relationtips *(edges)*
2.   building dataframes
3.   visualizing



## merge dataframes

### proED


In [ ]:
df_all_posts_proED, df_all_comments_proED = merge_dataframes(reddit_data_paths_proED)

In [ ]:
#indexing by id
df_all_comments_proED = df_all_comments_proED.set_index("id")
df_all_posts_proED = df_all_posts_proED.set_index("id")
df_all_comments_proED["id"] = df_all_comments_proED.index
df_all_posts_proED["id"] = df_all_posts_proED.index
#df_all_comments["crbogrx"]

In [ ]:
df_all_posts_proED

,author,title,selftext,num_comments,score,created_utc,subreddit,id
id,,,,,,,,
368gaa,Knyburg,Ice Blonde,None,2,22,1431837086,proED,368gaa
368gzz,Klairvoyant,The proED Diet/Workout Plan,"\n1. No simple carbs (ie sugar, white bread)\n...",0,5,1431837639,proED,368gzz
36akr0,Klairvoyant,2468 Diet,None,0,4,1431893321,proED,36akr0
375qar,Knyburg,Blue Eyes,None,0,15,1432521968,proED,375qar
3gcylw,AutoModerator,"Daily Food Diary! August 09, 2015",This is a daily food diary thread for August 0...,12,2,1439136152,proED,3gcylw
4sbqcp,lunaticsloverspoets,Baby's First EC Stack,You guys... I don't know what a cocaine high f...,12,16,1468253829,proED,4sbqcp
4sbsyz,HowToBeAsian25,Need some help determining goals,"I'm a very visual person (duh, I think we all ...",12,11,1468254642,proED,4sbsyz
4sbxbp,HowToBeAsian25,How to think about food.,"IDK if this will help anyone, but it is someth...",3,25,1468255950,proED,4sbxbp
4sc2t6,ImOkWhyDoYouAsk,[Recipe-lunch] Salad w/ Low-fat Cottage Cheese...,None,7,26,1468257610,proED,4sc2t6


In [ ]:
#df_all_posts_proED.to_csv('/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_7/df_all_posts_proED.csv')

###Delete the rows with bots as users

In [ ]:
with open('/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_6/redditbots.csv', 'r') as f:
    reader = csv.reader(f)
    bots_users = list(reader)

bots = []
for b in bots_users:
  bots.append(b[0].replace("\t", ''))

dfs = pd.DataFrame()
for name in bots:
  print(name)
  print(df_all_posts_proED[df_all_posts_proED.author == name])

A858DE45F56D9BC9
Empty DataFrame
Columns: [author, title, selftext, num_comments, score, created_utc, subreddit, id]
Index: []
AAbot
Empty DataFrame
Columns: [author, title, selftext, num_comments, score, created_utc, subreddit, id]
Index: []
ADHDbot
Empty DataFrame
Columns: [author, title, selftext, num_comments, score, created_utc, subreddit, id]
Index: []
ALTcointip
Empty DataFrame
Columns: [author, title, selftext, num_comments, score, created_utc, subreddit, id]
Index: []
AVR_Modbot
Empty DataFrame
Columns: [author, title, selftext, num_comments, score, created_utc, subreddit, id]
Index: []
A_random_gif
Empty DataFrame
Columns: [author, title, selftext, num_comments, score, created_utc, subreddit, id]
Index: []
AltCodeBot
Empty DataFrame
Columns: [author, title, selftext, num_comments, score, created_utc, subreddit, id]
Index: []
Antiracism_Bot
Empty DataFrame
Columns: [author, title, selftext, num_comments, score, created_utc, subreddit, id]
Index: []
ApiContraption
Empty DataFra

###Replace the deleted users

In [ ]:
df_all_posts_proED[df_all_posts_proED.author == '[deleted]']

,author,title,selftext,num_comments,score,created_utc,subreddit,id
id,,,,,,,,
4sczwu,[deleted],I've been emotionally abused my entire life by...,[deleted],2,14,1468267792,proED,4sczwu
4sd9aw,[deleted],Turns out telling yourself you won't care abou...,[deleted],1,9,1468270860,proED,4sd9aw
4sdaep,[deleted],My stomach is ridiculously bloated what could ...,"That's it, restricting or not, my stomach some...",12,6,1468271244,proED,4sdaep
4sdtiq,[deleted],I finally met someone else with an ED (that I ...,"There we were, having our first night out as f...",2,11,1468278006,proED,4sdtiq
3gd9s0,[deleted],Some real-life thinspo from Warped Tour,None,0,2,1439141821,proED,3gd9s0
4seb92,[deleted],Could anyone help me make my (hopefully) last ...,[deleted],0,1,1468284653,proED,4seb92
4sestz,[deleted],[Rant] Meh.,[deleted],2,7,1468291349,proED,4sestz
4sf4rd,[deleted],"Do you ever feel like you aren't sick ""enough""?",[deleted],14,28,1468296525,proED,4sf4rd
3gdexs,[deleted],[thinspo] White lights are too bright. I just ...,None,0,1,1439144372,proED,3gdexs


In [ ]:
''.join(random.choices(string.ascii_uppercase + string.digits, k=N))

NameError: ignored

In [ ]:
df_all_comments_proED

,author,created_utc,link_id,parent_id,score,body,subreddit,id
id,,,,,,,,
crbogrx,Klairvoyant,1431837901,t3_368hc0,t3_368hc0,4,Height: 5'6\n\nCW: 125.5\n\nHW: 138\n\nLW: 78....,proED,crbogrx
crbop6v,Knyburg,1431838552,t3_368hc0,t3_368hc0,6,height: 5'2\nCW: 100.5\nHW: 128.3\nLW: 88\nGW:...,proED,crbop6v
crccsum,Klairvoyant,1431902277,t3_36aa31,t1_crccp77,1,Sorry you feel that way. You are welcomed to d...,proED,crccsum
crgs5es,FascistAsparagus,1432243177,t3_36prv5,t3_36prv5,1,A lot of times what people think is fatigue fr...,proED,crgs5es
cry71h7,Alexa-the-hexa,1433654414,t3_38vl70,t1_cry70dl,0,My problem is mainly that I already eat not a ...,proED,cry71h7
cv74rby,byebyeb1rd,1442687947,t3_3lkim6,t3_3lkim6,1,**Breakfast:** Greek Yogurt-100 cal\n\n**Lunch...,proED,cv74rby
d6a8ueo,ifuckpineapples,1470742664,t3_4wtg6r,t3_4wtg6r,7,There were free munchkins at work and I had c&...,proED,d6a8ueo
d6a8v9c,Someone_Who_Isnt_You,1470742726,t3_4wtq5l,t1_d69zk0u,2,"Omg, we are the same size... Makes me want a l...",proED,d6a8v9c
d6a90a1,explodingeyeballs,1470743085,t3_4wtq5l,t1_d6a8v9c,2,Dude I'm the same size too!! Yeah I know right 😭,proED,d6a90a1


In [ ]:
#df_all_comments_proED.to_csv('/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_7/df_all_comments_proED.csv')

In [ ]:
df_all_comments_proED[df_all_comments_proED.author == '[deleted]']

In [ ]:
with open('/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_6/redditbots.csv', 'r') as f:
    reader = csv.reader(f)
    bots_users = list(reader)

bots = []
for b in bots_users:
  bots.append(b[0].replace("\t", ''))

dfs = pd.DataFrame()
for name in bots:
  print(name)
  print(df_all_comments_proED[df_all_comments_proED.author == name])

### why ?
* we now only need to work with ids and find relationships btwn them
* we it will help establish a network using ids only
* edges' weights will feed on the info provided by the from_id, to_id

## charge info into dict

In [ ]:
post_comments_proED, comment_replies_proED, author_posts_proED, author_comments_proED = initialize_lists(df_all_posts_proED , df_all_comments_proED, {}, {}, {}, {})

In [ ]:
post_comments_proED, comment_replies_proED, author_posts_proED, author_comments_proED = asign_child_to_parent(df_all_posts_proED, df_all_comments_proED, post_comments_proED, comment_replies_proED, author_posts_proED, author_comments_proED)

In [ ]:
print( "number of posters     : " + str(len(author_posts_proED)))
print( "number of commenters  : " + str(len(author_comments_proED)))
print( "number of posts       : " + str(len(post_comments_proED)))
print( "number of comments    : " + str(len(comment_replies_proED)))

number of posters     : 3652
number of commenters  : 7135
number of posts       : 26867
number of comments    : 245164


In [ ]:
author_comments_proED.keys



<function dict.keys>

In [ ]:
for ind2, row2 in author_comments_proED:
    if row['author'] != row2['author']:
      users_ed.append(row['author'])



## transition to users relationships

In [ ]:
poster_commenters_proED = get_poster_commenters_proED(author_posts_proED, post_comments_proED)
commenter_repliers_proED = get_commenter_repliers_proED(author_comments_proED, comment_replies_proED)

###proRecovery


In [ ]:
df_all_posts_proRecov, df_all_comments_proRecov = merge_dataframes(reddit_data_paths_proRecov)

In [ ]:
#indexing by id
df_all_comments_proRecov = df_all_comments_proRecov.set_index("id")
df_all_posts_proRecov = df_all_posts_proRecov.set_index("id")
df_all_comments_proRecov["id"] = df_all_comments_proRecov.index
df_all_posts_proRecov["id"] = df_all_posts_proRecov.index
#df_all_comments["crbogrx"]

In [ ]:
df_all_posts_proRecov

,author,title,selftext,num_comments,score,created_utc,subreddit,id
id,,,,,,,,
9x60kr,pacific_rimmed,Feeling so untethered right now - would anyone...,,17,1,1542242204,EDAnonymous,9x60kr
9x60wj,thingsarestranger,Something positive you learned/experienced fro...,I had always considered laxatives as a form of...,95,1,1542242281,EDAnonymous,9x60wj
9x6p1a,AnnaGreen3,Can we do something?,Is there something we can do to recover r/proE...,10,1,1542247372,EDAnonymous,9x6p1a
9xcxo1,taikutsuu,Please recognize that there is a realistic cha...,"Just yesterday, one other subreddit was taken ...",106,1,1542300970,EDAnonymous,9xcxo1
9zu8wo,macespacee,Depression medication and appetite// weight lo...,I have a really bad depression and depression ...,11,1,1543023643,EDAnonymous,9zu8wo
b540kd,raspbunni,reached a new hw :):):):),i stuffed myself like a fat pig while pregnant...,14,39,1553475124,EDAnonymous,b540kd
b540m2,probablyfuckedurdad,I wish people were more concerned about me. Is...,I told my parents about my ed when I put mysel...,5,57,1553475131,EDAnonymous,b540m2
b54duo,citrus_juuiice,I ate normal for two days,"I feel so gross, I ate like 1500-2000 calories...",1,23,1553477232,EDAnonymous,b54duo
b54ewh,lovedieshere,Overeater finally became an undereater...,"So I've had an EDNOS for years, but always on ...",7,15,1553477397,EDAnonymous,b54ewh


In [ ]:
#df_all_posts_proRecov.to_csv('/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_7/df_all_posts_proRecov.csv')

In [ ]:
with open('/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_6/redditbots.csv', 'r') as f:
    reader = csv.reader(f)
    bots_users = list(reader)

bots = []
for b in bots_users:
  bots.append(b[0].replace("\t", ''))

dfs = pd.DataFrame()
for name in bots:
  print(name)
  print(df_all_posts_proRecov[df_all_posts_proRecov.author == name])

In [ ]:
dfs

In [ ]:
df_all_posts_proRecov[df_all_posts_proRecov.author == '[deleted]']

In [ ]:
df_all_comments_proRecov

In [ ]:
df_all_comments_proRecov.to_csv('/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_7/df_all_comments_proRecov.csv')

In [ ]:
df_all_comments_proRecov[df_all_comments_proRecov.author == '[deleted]']

In [ ]:
with open('/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_6/redditbots.csv', 'r') as f:
    reader = csv.reader(f)
    bots_users = list(reader)

bots = []
for b in bots_users:
  bots.append(b[0].replace("\t", ''))

dfs = pd.DataFrame()
for name in bots:
  print(name)
  print(df_all_comments_proRecov[df_all_comments_proRecov.author == name])

In [ ]:
dfs

## charge info into dict

In [ ]:
post_comments_proRecov, comment_replies_proRecov, author_posts_proRecov, author_comments_proRecov = initialize_lists(df_all_posts_proRecov , df_all_comments_proRecov, {}, {}, {}, {})

In [ ]:
post_comments_proRecov, comment_replies_proRecov, author_posts_proRecov, author_comments_proRecov = asign_child_to_parent(df_all_posts_proRecov, df_all_comments_proRecov, post_comments_proRecov, comment_replies_proRecov, author_posts_proRecov, author_comments_proRecov)

In [ ]:
print( "number of posters     : " + str(len(author_posts_proRecov)))
print( "number of commenters  : " + str(len(author_comments_proRecov)))
print( "number of posts       : " + str(len(post_comments_proRecov)))
print( "number of comments    : " + str(len(comment_replies_proRecov)))

number of posters     : 3391
number of commenters  : 7491
number of posts       : 11402
number of comments    : 89028


## transition to users relationships

In [ ]:
poster_commenters_proRecov = get_poster_commenters_proRecov(author_posts_proRecov, post_comments_proRecov)
commenter_repliers_proRecov = get_commenter_repliers_proRecov(author_comments_proRecov, comment_replies_proRecov)

_

_

_

# Drawing the graph

## build df [ [ "from",  "to"]  ]

### replies, comments and posts as NODES

#proED

In [ ]:
l_proED = []
for key, value in post_comments_proED.items():
  for c in value:
    l_proED.append((c, key))

In [ ]:
df_comment_to_post_proED = pd.DataFrame(l_proED, columns = ['from', 'to'])

In [ ]:
ll_proED = []
for key, value in comment_replies_proED.items():
  for c in value:
    ll_proED.append((c, key))

In [ ]:
df_reply_to_comment_proED = pd.DataFrame(ll_proED, columns = ['from', 'to'])

### users as NODES

In [ ]:
l_proED = []
for key, value in poster_commenters_proED.items():
  for c in value:
    l_proED.append((c, key))

In [ ]:
df_comment_to_post_proED = pd.DataFrame(l_proED, columns = ['from', 'to'])

In [ ]:
df_comment_to_post_proED

In [ ]:
ll_proED = []
for key, value in commenter_repliers_proED.items():
  for c in value:
    ll_proED.append((c, key))

In [ ]:
df_reply_to_comment_proED = pd.DataFrame(ll_proED, columns = ['from', 'to'])

In [ ]:
df_reddit_proED = df_comment_to_post_proED.append(df_reply_to_comment_proED, ignore_index=True)

In [ ]:
df_reddit_proED

In [ ]:
df_reddit_proED.to_csv('/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_7/df_reddit_proED.csv')

#proRecovery


In [ ]:
l_proRecov = []
for key, value in post_comments_proRecov.items():
  for c in value:
    l_proRecov.append((c, key))

In [ ]:
df_comment_to_post_proRecov = pd.DataFrame(l_proRecov, columns = ['from', 'to'])

In [ ]:
ll_proRecov = []
for key, value in comment_replies_proRecov.items():
  for c in value:
    ll_proRecov.append((c, key))

In [ ]:
df_reply_to_comment_proRecov = pd.DataFrame(ll_proRecov, columns = ['from', 'to'])

###users as NODES

In [ ]:
l_proRecov = []
for key, value in poster_commenters_proRecov.items():
  for c in value:
    l_proRecov.append((c, key))

In [ ]:
df_comment_to_post_proRecov = pd.DataFrame(l_proRecov, columns = ['from', 'to'])

In [ ]:
df_comment_to_post_proRecov

In [ ]:
ll_proRecov = []
for key, value in comment_replies_proRecov.items():
  for c in value:
    ll_proRecov.append((c, key))

In [ ]:
df_reply_to_comment_proRecov = pd.DataFrame(ll_proRecov, columns = ['from', 'to'])

In [ ]:
df_reddit_proRecov = df_comment_to_post_proRecov.append(df_reply_to_comment_proRecov, ignore_index=True)

In [ ]:
df_reddit_proRecov

In [ ]:
df_reddit_proRecov.to_csv('/content/gdrive/My Drive/Studies/Projects/PFE/PFE/Week_7/df_reddit_proRecov.csv')

## draw

In [ ]:
G_directed = nx.from_pandas_edgelist(df_reddit, 'from', 'to', create_using=nx.DiGraph())

##Macro level analysis proED


In [ ]:
density = nx.density(G_directed)
print("Network density:", density)

In [ ]:
triadic_closure = nx.transitivity(G_directed)
print("Triadic closure:", triadic_closure)

##Macro level analysis proRecov

In [ ]:
density = nx.density(G_directed)
print("Network density:", density)

In [ ]:
triadic_closure = nx.transitivity(G_directed)
print("Triadic closure:", triadic_closure)

# **Micro level analysis**

In [ ]:
degree_dict = dict(G_directed.degree(G_directed.nodes()))

In [ ]:
betweenness_dict = nx.betweenness_centrality(G_directed)

In [ ]:
closeness_dict = nx.closeness_centrality(G_directed)

In [ ]:
eigenvector_dict = nx.eigenvector_centrality(G_directed)

In [ ]:
centralities = pd.DataFrame.from_dict([degree_dict, closeness_dict, betweenness_dict, eigenvector_dict])

In [ ]:
rownames=['degree centrality', 'closeness centrality', 'betweenness centrality', 'eigenvector centrality']
centralities.index = rownames

In [ ]:
centralities = centralities.transpose()

In [ ]:
centralities